In [ ]:
pip install streamlit

In [ ]:
pip install langchain langchain-community

In [ ]:
pip install accelerate

In [ ]:
!pip install --upgrade pandas==2.2.2 networkx==3.2


In [ ]:
pip install TTS


In [ ]:
pip install bitsandbytes

In [ ]:
pip install --upgrade numpy transformers


In [ ]:
pip install --upgrade torch


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import IPython.display as ipd
from langchain import HuggingFacePipeline, LLMChain
from langchain.prompts import PromptTemplate,StringPromptTemplate
from transformers import pipeline as tts_pipeline
from IPython.display import Audio


In [ ]:
access_token = " "#insert your hugging face access token here
import os
HUGGINGFACE_API_TOKEN = os.getenv(" ")#insert your hugging face access token here
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        token=access_token,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_new_tokens=500,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto",
    temperature=1.0,  # Controls creativity. Higher value = more random.
    top_k=50,         # Considers top 50 tokens for the next word.
    top_p=0.9         # Nucleus sampling. Picks from smallest set with 90% probability mass.
)


In [ ]:
from typing import Any
llm = HuggingFacePipeline(pipeline=pipe)

# Function to generate text from an image
def generate_text_from_image(url: str) -> str:
    image_to_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
    generated_text = image_to_text(url)[0]["generated_text"]
    return generated_text



In [ ]:
import random
# Function that generates a random story based on image text and optional user input
def generate_story(image_url: str, user_input: str = "") -> str:
    """
    Generates a complete story from the image and optional user input.
    The story length is now flexible and won't be cut off.

    :param image_url: URL of the image
    :param user_input: Optional input to enhance the story
    :return: Generated story
    """
    # Generate text from the image
    image_text = generate_text_from_image(image_url)

   # Add a random twist or element to the prompt to ensure variability
    random_twist = random.choice([
        "a new friend is discovered",
        "a hidden treasure is found",
        "the characters meet a helpful animal",
        "everyone learns a surprising dance move",
        "a magical rainbow appears",
        "a funny creature joins the adventure",
        "a friendly wizard grants a wish",
        "they find a secret map leading to fun places",
        "a magical garden grows around them",
        "a playful wind brings floating bubbles"
    ])
    # Define the prompt template based on whether user input is provided
    if user_input:
        template = f"""
        You are a talented storyteller who creates a story from a simple narrative.
        Create a unique and complete story using the following scenario. The story should have an unpredictable twist.

        CONTEXT: {{scenario}}
        USER INPUT: {{user_input}}
        RANDOM TWIST: {random_twist}
        STORY:
        """
        input_variables = ['scenario', 'user_input']
        prompt = PromptTemplate(template=template, input_variables=input_variables)
        # Generate the story using both image text and user input
        story = LLMChain(llm=llm, prompt=prompt, verbose=True)({"scenario": image_text, "user_input": user_input})['text']
    else:
        template = f"""
        You are a talented storyteller who creates a story from a simple narrative.
        Create a unique and complete story using the following scenario. The story should include an unpredictable twist.

        CONTEXT: {{scenario}}
        RANDOM TWIST: {random_twist}
        STORY:
        """
        input_variables = ['scenario']
        prompt = PromptTemplate(template=template, input_variables=input_variables)
        # Generate the story using only the image text
        story = LLMChain(llm=llm, prompt=prompt, verbose=True)({"scenario": image_text})['text']

    # Extract the generated story
    story = story.split('STORY:')[-1].strip()

    # Return the story
    return story

In [ ]:
# Example usage
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTK1-jnmxzUJLYT1XGcVC21gN8QC17ZjGZALA&s"
user_input = " "



In [ ]:
import cv2
import numpy as np
import requests
import matplotlib.pyplot as plt
from io import BytesIO

image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTK1-jnmxzUJLYT1XGcVC21gN8QC17ZjGZALA&s"

# Fetch the image
response = requests.get(image_url)
image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

if image is None:
    print("Error: Failed to load image from URL!")
else:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.axis("off")
    plt.show()


In [ ]:
# Generate story
import textwrap
story = generate_story(image_url, user_input)
width_story=textwrap.fill(story,width=100)
print("Generated Story:", width_story)



In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.23.5


In [ ]:
!pip uninstall librosa -y
!pip install librosa


In [ ]:


# Load the Bark TTS pipeline
pipe_tts = pipeline("text-to-speech", model="suno/bark")

def generate_speech_from_story(story: str):
    """
    Converts a generated story to speech using Hugging Face Bark TTS pipeline.

    :param story: The generated story text
    :return: Audio output
    """
    # Generate audio from text
    output = pipe_tts(story)

    # Play the audio
    return ipd.Audio(output["audio"], rate=22050)

# Example usage with an LLM-generated story
#story = story

# Generate speech
audio_output = generate_speech_from_story(story)

# Play the audio
audio_output


In [ ]:
'''# Function to generate speech from text
def generate_speech_from_story(story: str):
    """
    Converts a generated story to speech.

    :param story: The generated story text
    :return: Audio output
    """
    # Create TTS pipeline
    pipe_tts = tts_pipeline("text-to-speech", model="suno/bark-small")
    output_audio = pipe_tts(story)

    # Play the audio
    return output_audio["audio"]

'''

In [ ]:
'''import torch
from TTS.api import TTS
import IPython.display as ipd  # For playing audio in Jupyter

# Load a TTS model (choose from Coqui TTS available models)
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC")  # Example model


'''

In [ ]:
'''# Function to convert a story to speech
def generate_speech_from_story(story):
    """
    Converts a generated story to speech and plays the audio.

    :param story_text: The generated story text
    """
    output_path = "output_story.wav"

    # Generate speech from the text and save to a file
    tts.tts_to_file(text=story, file_path=output_path,max_decoder_steps=20000)

    # Play the generated audio in Jupyter Notebook
    return ipd.Audio(output_path, rate=22050)

# Example: Using a generated story
generated_story = story

# Convert story to speech and play
audio_output = generate_speech_from_story(generated_story)
audio_output  # This will play the audio in Jupyter Notebook'''

In [ ]:
'''from TTS.api import TTS

# Create an instance of TTS first
tts = TTS()

# Now list available models
print(tts.list_models())
'''

In [ ]:
# Generate speech
#audio_output = generate_speech_from_story(story)



In [ ]:
# Play the audio
#Audio(audio_output, rate=22050)